# How good was my strategy?

In [ ]:
environment = "PAPER" # PAPER / PROD / BACKTEST
date = "2020-12-17"
strategy ='short_trap_buster'

### imports

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from pytz import timezone
import matplotlib.pyplot as plt
from IPython.display import HTML, display, Markdown
from liualgotrader.models.gain_loss import TradeAnalysis
from liualgotrader.models.trending_tickers import TrendingTickers
from liualgotrader.analytics.analysis import strategy_return_for_qty_one
import alpaca_trade_api as tradeapi

### connect to data-source

In [ ]:
api = tradeapi.REST(base_url="https://api.alpaca.markets")

## How to calculate strategy relevance, using p-value

* Pick $e𝑛𝑣𝑖𝑟𝑜𝑛𝑚𝑒𝑛𝑡$ , $𝑠𝑡𝑟𝑎𝑡𝑒𝑔𝑦$ , and a $𝑑𝑎𝑡𝑒$ when strategy was used,
* Let $window$ be the length in minutes of trading windows, during which $strategy$ may buy a stock,
* Let $C$ be the list of all scanned stocks during $date$,
* For $c \in C$, Let $T_{0}(c)$=the time $c$ was added to $C$, 
* For $c \in C$, Let $Duration(c)$=$window-T_{0}(c)$, 
* For $c \in C$, Let $Hold(c)$=sum of time $stategy$ had position in $c$, 
* Let $buy-actions$/$sell-actions$ represent all buy(/sell) decisions made by $strategy$,

### $Prob(BUY) = \frac{|buy-actions|}{\sum \limits _{c}^{C} Duration(c) - Hold(c)}$

### $Prob(SELL) = \frac{|sell-actions|}{\sum \limits _{c}^{C} Hold(c)}$

### pseudo-code

<code>gain-vs-strat = 0 
    for i in range(10000):
        gain = 0 
        for c in C:
            for t in range($T_{0}(c)$,$windows$):
                gain += calculate gain based on Prob(BUY) and Prob(SELL)
        gain-vs-strat += 1 if stategy_gain > gain else 0</code>      

## $1 - p_{value} = \frac{gain-vs-strat}{10,000}$

NOTE: below 0.95 strategy is bad. 

## The real stuff

### calculate gains from my strategy

In [ ]:
my_gains = await strategy_return_for_qty_one(
    strategy=strategy, env=environment, start_date=datetime.strptime(date, "%Y-%m-%d")
)
my_gains

In [ ]:
scanned = await TrendingTickers.load_by_date_and_env(env=environment, start_date=datetime.strptime(date, "%Y-%m-%d"))
scanned['hold'] = timedelta(0)
print(f"loaded {len(scanned)} entries")

In [ ]:
df = await TradeAnalysis.load(environment,  datetime.strptime(date, "%Y-%m-%d"))
df = df.loc[df.algo_name == strategy]
print(f"loaded {len(df)} entries")

In [ ]:
for i, row in scanned.iterrows():
    hold_time = df.loc[
        (df.batch_id == row.batch_id) & (df.symbol == row.symbol)
    ].hold.sum()
    scanned.loc[scanned.index == i, "hold"] = (
        hold_time if not pd.isnull(hold_time) else timedelta(0)
    )
    duration = timedelta(minutes=120) - (
        row.create_tstamp.to_pydatetime()
        - row.create_tstamp.to_pydatetime().replace(
            hour=14, minute=30, second=0, microsecond=0
        )
    )
    scanned.loc[scanned.index == i, "duration"] = (
        duration if duration >= timedelta(days=0) else pd.NaT
    )

In [ ]:
scanned=scanned.dropna(how='any',axis=0) 
scanned['delta'] = scanned.duration-scanned.hold

In [ ]:
prob_buy = 1.0 * len(df) / (scanned.delta.sum().total_seconds() // 60)
prob_buy

In [ ]:
prob_sell = 1.0 * len(df) / (scanned.hold.sum().total_seconds() // 60)
prob_sell

In [ ]:
def gen_rand(prob:float) -> int:
    return np.random.choice([1,0],1,p=[prob,1-prob])[-1]

In [ ]:
utc = timezone("UTC")

In [ ]:
minute_history = {}

In [ ]:
def load_data(symbol: str, start: datetime, end: datetime) -> pd.DataFrame:
    print("load", symbol, start)
    retry = 5
    while retry > 0:
        try:
            payload = api.polygon.historic_agg_v2(
                symbol,
                1,
                "minute",
                _from=str(start),
                to=str(end),
            ).df
            break
        except Exception:
            retry -= 1
            continue
            
    return payload


def calculate_random_strategy():
    gain = 0
    for i, row in scanned.iterrows():
        b = row.create_tstamp.replace(second=0, microsecond=0)
        end_buy = b.replace(hour=16, minute=30, second=0, microsecond=0)
        end_sell = b.replace(hour=21, minute=0, second=0, microsecond=0)
        while b < end_buy:
            to_buy = False
            if gen_rand(prob_buy):
                # print(f"found buy {row.symbol} {b}")
                to_buy = True

            b += timedelta(minutes=1)
            if to_buy:
                s = b
                while s < end_sell:
                    if gen_rand(prob_sell):
                        break
                    s += timedelta(minutes=1)
                    
                if row.symbol not in minute_history:
                    minute_history[row.symbol] = load_data(row.symbol, b, s)
                else:
                    try:
                        _ = minute_history[row.symbol].index.get_loc(
                            str(pd.Timestamp(b, tz=utc).tz_convert("US/Eastern")),
                            method="pad",
                            tolerance=None,
                        )
                    except Exception as e:
                        new_data = load_data(row.symbol, b, s)

                        minute_history[row.symbol] = (
                            minute_history[row.symbol].append(new_data)
                            if minute_history[row.symbol].index[0] < new_data.index[0]
                            else new_data.append(minute_history[row.symbol])
                        )

                b_index = minute_history[row.symbol].index.get_loc(
                    str(pd.Timestamp(b, tz=utc).tz_convert("US/Eastern")),
                    method="pad",
                    tolerance=None,
                )
                s_index = minute_history[row.symbol].index.get_loc(
                    str(pd.Timestamp(s, tz=utc).tz_convert("US/Eastern")),
                    method="pad",
                    tolerance=None,
                )
                gain += (
                    minute_history[row.symbol].iloc[s_index].close
                    - minute_history[row.symbol].iloc[b_index].close
                )

                b = s
    return gain

In [ ]:
i_win = 0
for i in range (10000):
    random_gains = calculate_random_strategy()
    if my_gains > random_gains:
        i_win += 1
        
    print(i+1, i_win/(i+1), random_gains)